In [1]:
import os
import time
import re
import numpy as np 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
import spacy
import pickle
nltk.download('stopwords')
import unidecode
from sklearn.utils import shuffle
import sklearn.metrics as met
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
import gensim

[nltk_data] Downloading package stopwords to /home/thomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from IPython.display import display,HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_colwidth",200)

In [3]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [4]:
startTime = time.time()

# Data

In [5]:
path = '/home/thomas/Documents/openclassroom/projet5/'
path_res = path +'results/'

In [6]:
with open(path + 'cleanDataprepSmote3.pickle','rb') as f:
    data = pickle.load(f)

In [7]:
data.shape

(51904, 20)

In [8]:
data.dtypes

Title                        object
Body                         object
Tags                         object
text                         object
nb_char                       int64
tokens                       object
tokens_title                 object
nb_tokens                     int64
tokens_unique                object
nb_tokens_unique              int64
tokens_WSW                   object
tokens_title_WSW             object
nb_tokens_WSW                 int64
tokens_lemma                 object
len_tokens_lemma              int64
Tags_list                    object
len _Tags_list                int64
filtered_tags                object
filtered_tags_reduced        object
len_filtered_tags_reduced     int64
dtype: object

In [9]:
data.head()

,Title,Body,Tags,text,nb_char,tokens,tokens_title,nb_tokens,tokens_unique,nb_tokens_unique,tokens_WSW,tokens_title_WSW,nb_tokens_WSW,tokens_lemma,len_tokens_lemma,Tags_list,len _Tags_list,filtered_tags,filtered_tags_reduced,len_filtered_tags_reduced
0,transpose/unzip function (inverse of zip)?,<p>I have a list of 2-item tuples and I'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.</p>\n\n<p><strong>For exa...,<python><list><matrix><transpose>,i have a list of 2-item tuples and i'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.\nfor example:\noriginal = [(...,350,"[i, have, a, list, of, 2, item, tuples, and, i, d, like, to, convert, them, to, 2, lists, where, the, first, contains, the, first, item, in, each, tuple, and, the, second, list, holds, the, second...","[transpose, unzip, function, inverse, of, zip]",70,"(have, d, list, and, result, to, lists, c, convert, second, example, 2, for, original, 4, holds, want, builtin, #, there, item, them, that, tuples, contains, where, first, in, function, does, like...",42,"[list, item, tuples, like, convert, lists, first, contains, first, item, tuple, second, list, holds, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]","[transpose, unzip, function, inverse, zip]",28,"[list, item, tuple, like, convert, list, first, contain, first, item, tuple, second, list, hold, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]",28,"[list, transpose, matrix, python]",4,"[list, python]",tag0,1
1,detecting audio silence in wav files using c#,"<p>I'm tasked with building a .NET client app to detect silence in a WAV files.</p>\n\n<p>Is this possible with the built-in Windows APIs? Or alternately, any good libraries out there to help with...",<c#><.net><audio>,"i'm tasked with building a .net client app to detect silence in a wav files.\nis this possible with the built-in windows apis? or alternately, any good libraries out there to help with this?\n",190,"[i, m, tasked, with, building, a, net, client, app, to, detect, silence, in, a, wav, files, is, this, possible, with, the, built, in, windows, apis, or, alternately, any, good, libraries, out, the...","[detecting, audio, silence, in, wav, files, using, c#]",36,"(good, tasked, silence, to, windows, app, files, this, help, alternately, any, there, is, wav, built, detect, m, in, building, possible, libraries, or, apis, out, with, a, the, i, net, client)",30,"[tasked, building, net, client, app, detect, silence, wav, files, possible, built, windows, apis, alternately, good, libraries, help]","[detecting, audio, silence, wav, files, using, c#]",17,"[task, build, net, client, app, detect, silence, wav, file, possible, build, window, apis, alternately, good, library, help]",17,"[c#, audio, .net]",3,"[c#, .net]",tag1,1
2,how to request a random row in sql?,<p>How can I request a random row (or as close to truly random as is possible) in pure SQL?</p>\n,<sql><random>,how can i request a random row (or as close to truly random as is possible) in pure sql?\n,89,"[how, can, i, request, a, random, row, or, as, close, to, truly, random, as, is, possible, in, pure, sql]","[how, to, request, a, random, row, in, sql]",19,"(a, row, how, i, sql, in, to, close, can, truly, request, possible, is, or, random, pure, as)",17,"[request, random, row, close, truly, random, possible, pure, sql]","[request, random, row, sql]",9,"[request, random, row, close, truly, random, possible, pure, sql]",9,"[random, sql]",2,[sql],tag108,1
4,detecting current iphone input language,"<p>Does anybody knows, can I get the current input language and/or keyboard layout in iPhone application? Can I also get a notification when input language was changed?</p>\n",<iphone><keyboard>,"does anybody knows, can i get the current input language and/or keyboard layout in

## prepare data

list all empty filtered_tags

In [10]:
data['filtered_tags'][data['filtered_tags'].str.len() == 0]

Series([], Name: filtered_tags, dtype: object)

In [11]:
data['filtered_tags']=data['filtered_tags'].apply(lambda x : ['#empty'] if len(''.join(x)) == 0 else x)

In [12]:
data['filtered_tags'][data['filtered_tags'].str.len() == 0]

Series([], Name: filtered_tags, dtype: object)

In [13]:
print(data.shape)

(51904, 20)


In [14]:
data.head(8)

,Title,Body,Tags,text,nb_char,tokens,tokens_title,nb_tokens,tokens_unique,nb_tokens_unique,tokens_WSW,tokens_title_WSW,nb_tokens_WSW,tokens_lemma,len_tokens_lemma,Tags_list,len _Tags_list,filtered_tags,filtered_tags_reduced,len_filtered_tags_reduced
0,transpose/unzip function (inverse of zip)?,<p>I have a list of 2-item tuples and I'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.</p>\n\n<p><strong>For exa...,<python><list><matrix><transpose>,i have a list of 2-item tuples and i'd like to convert them to 2 lists where the first contains the first item in each tuple and the second list holds the second item.\nfor example:\noriginal = [(...,350,"[i, have, a, list, of, 2, item, tuples, and, i, d, like, to, convert, them, to, 2, lists, where, the, first, contains, the, first, item, in, each, tuple, and, the, second, list, holds, the, second...","[transpose, unzip, function, inverse, of, zip]",70,"(have, d, list, and, result, to, lists, c, convert, second, example, 2, for, original, 4, holds, want, builtin, #, there, item, them, that, tuples, contains, where, first, in, function, does, like...",42,"[list, item, tuples, like, convert, lists, first, contains, first, item, tuple, second, list, holds, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]","[transpose, unzip, function, inverse, zip]",28,"[list, item, tuple, like, convert, list, first, contain, first, item, tuple, second, list, hold, second, item, example, original, b, c, #, want, become, result, b, c, builtin, function]",28,"[list, transpose, matrix, python]",4,"[list, python]",tag0,1
1,detecting audio silence in wav files using c#,"<p>I'm tasked with building a .NET client app to detect silence in a WAV files.</p>\n\n<p>Is this possible with the built-in Windows APIs? Or alternately, any good libraries out there to help with...",<c#><.net><audio>,"i'm tasked with building a .net client app to detect silence in a wav files.\nis this possible with the built-in windows apis? or alternately, any good libraries out there to help with this?\n",190,"[i, m, tasked, with, building, a, net, client, app, to, detect, silence, in, a, wav, files, is, this, possible, with, the, built, in, windows, apis, or, alternately, any, good, libraries, out, the...","[detecting, audio, silence, in, wav, files, using, c#]",36,"(good, tasked, silence, to, windows, app, files, this, help, alternately, any, there, is, wav, built, detect, m, in, building, possible, libraries, or, apis, out, with, a, the, i, net, client)",30,"[tasked, building, net, client, app, detect, silence, wav, files, possible, built, windows, apis, alternately, good, libraries, help]","[detecting, audio, silence, wav, files, using, c#]",17,"[task, build, net, client, app, detect, silence, wav, file, possible, build, window, apis, alternately, good, library, help]",17,"[c#, audio, .net]",3,"[c#, .net]",tag1,1
2,how to request a random row in sql?,<p>How can I request a random row (or as close to truly random as is possible) in pure SQL?</p>\n,<sql><random>,how can i request a random row (or as close to truly random as is possible) in pure sql?\n,89,"[how, can, i, request, a, random, row, or, as, close, to, truly, random, as, is, possible, in, pure, sql]","[how, to, request, a, random, row, in, sql]",19,"(a, row, how, i, sql, in, to, close, can, truly, request, possible, is, or, random, pure, as)",17,"[request, random, row, close, truly, random, possible, pure, sql]","[request, random, row, sql]",9,"[request, random, row, close, truly, random, possible, pure, sql]",9,"[random, sql]",2,[sql],tag108,1
4,detecting current iphone input language,"<p>Does anybody knows, can I get the current input language and/or keyboard layout in iPhone application? Can I also get a notification when input language was changed?</p>\n",<iphone><keyboard>,"does anybody knows, can i get the current input language and/or keyboard layout in

## shared functions

In [15]:
def wordFrequency(data):
    ''' take pandas series and count words retun the occurences of the elements'''
    freqTotal = {}
    for text in data:
        freq = nltk.Counter(text)
        for word, count in freq.items():
            if word in freqTotal.keys():
                freqTotal[word] += count
            else:
                freqTotal[word] = count
    return freqTotal

def splitData(df,XCol,Ycol,ratioToKeep=1):
    """ split data into two sets X and Y and then split them into  trainning and validation sets 
    the total will be 4 sets. the function returns pandas series or pandas dataframe types"""# add sample 
    df = df.sample(frac=ratioToKeep)
    print(f'data used shape: {df.shape}')
    X=df.loc[:,XCol]
    y=df.loc[:,Ycol]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    print(f'X_train types:\t{type(X_train)}')
    print(f'X_test types:\t{type(X_test)}')
    print(f'Y_test types:\t{type(y_test)}')
    print(f'Y_train types:\t{type(y_train)}')
    return X_train, X_test, y_train, y_test

def reduceData (Xtrain, Ytrain, ratioToKeep=0.2):
    size = Xtrain.shape[0]
    nbRowsToKeep = int(np.rint(size*ratioToKeep))
    indRowToKeep = np.random.choice(size, nbRowsToKeep)
    Xtrain = Xtrain[indRowToKeep,:]
    Ytrain = Ytrain[indRowToKeep,:]
    return (Xtrain,Ytrain)

In [16]:
def BOW(Xtrain, Xval):
    """ set the BOW method for X train and validation sets."""
    Xtrain = Xtrain.apply(lambda x : ' '.join(x))
    Xval = Xval.apply(lambda x : ' '.join(x))
    XtrainCorpus = [x for x in Xtrain]
    XvalCorpus = [x for x in Xval]
    vectorizer = CountVectorizer()
    Xtrain = vectorizer.fit_transform(XtrainCorpus)
    print('-'*15)
    print(f"Xtrain vectorized shape: {Xtrain.shape}\n")
    Xval = vectorizer.transform(XvalCorpus)
    print(f"Xval vectorized shape: {Xval.shape}\n")
    return (Xtrain,Xval)

def Binarize(Ytrain,Yval):
    """  binarize The tags for Y train and validation sets. """
    mlb = MultiLabelBinarizer()
    mlb.fit([Ytrain]) # if Y is not list of list, mlb.fit(['sci-fi', 'thriller', 'comedy']) -> array(['-', 'c', 'd', 'e', 'f', 'h', 'i', 'l', 'm', 'o', 'r', 's', 't','y'], dtype=object)
    for i in Ytrain.index:
        Ytrain.loc[i] = mlb.transform([[Ytrain.loc[i]]])[0]
    for i in Yval.index:
        Yval.loc[i] = mlb.transform([[Yval.loc[i]]])[0]
    labels = mlb.classes_
    print('-'*15)
    print(f"Ytrain vectorized shape: {Ytrain.shape}\n")
    print(f"Yval vectorized shape: {Yval.shape}\n")
    print(f"classes : {mlb.classes_}")
    return (Ytrain,Yval, labels)

def plotRoc():
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(labels)):
        fpr[i], tpr[i], _ = roc_curve(y_val[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    plt.figure()
    lw = 2
    plt.plot(
        fpr[2],
        tpr[2],
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc[2],
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()





def clfScore(model,Xval, Yval, labels, fileName,PATH=path_res):                           #  correct position of the labels ???
    y_pred = model.predict(Xval)
    F1score=met.f1_score(y_pred, Yval, average=None)
    preciScore=met.precision_score(y_pred, Yval, average=None)
    recallScore=met.recall_score(y_pred, Yval, average=None)
    print(f'{"Tags":25}\t\t F-score\t\t precision\t\t recall\n')
    with open(PATH + fileName,'w') as f:
        for ind,name in enumerate(labels):
            f.write(f'{name:25}\t\t {F1score[ind]:4f}\t\t {preciScore[ind]:4f}\t\t {recallScore[ind]:4f}\n')
            print(f'{name:25}\t\t {F1score[ind]:4f}\t\t {preciScore[ind]:4f}\t\t {recallScore[ind]:4f}')
    #plotRoc(labels,y_pred,y_val)
    
        

    
    

# WordTOvec test

In [17]:
### test  Word2vec features

In [18]:
X_train, X_val, y_train, y_val = splitData(data,XCol='tokens_lemma',Ycol='filtered_tags',ratioToKeep=0.5) # too much time without sampling

data used shape: (25952, 20)
X_train types:	<class 'pandas.core.series.Series'>
X_test types:	<class 'pandas.core.series.Series'>
Y_test types:	<class 'pandas.core.series.Series'>
Y_train types:	<class 'pandas.core.series.Series'>


In [19]:
# Xtrain, Xval = BOW(X_train, X_val)

In [20]:
Xtrain = X_train.to_list()
#display(Xtrain)

In [21]:
w2v_size=300
w2v_window=5
w2v_min_count=1
w2v_epochs=100
maxlen = 24 # adapt to length of sentences
#sentences = [gensim.utils.simple_preprocess(text) for text in Xtrain]
#sentences = [gensim.utils.simple_preprocess(Xtrain)]
sentences = Xtrain

In [22]:
# Création et entraînement du modèle Word2Vec

print("Build & train Word2Vec model ...")
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)
#                                               workers=multiprocessing.cpu_count())
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
#w2v_model.corpus_count -> Xtrain size
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Build & train Word2Vec model ...
Vocabulary size: 46783
Word2Vec trained


In [23]:
w2v_model.wv.most_similar(positive=['python','dataframe'], topn=5)

[('panda', 0.35724255442619324),
 ('one', 0.3434944748878479),
 ('array', 0.3403604030609131),
 ('c', 0.33116263151168823),
 ('list', 0.32976242899894714)]

In [24]:
#w2v_model.wv['python'] + w2v_model.wv['dataframe']
#w2v_model.wv.most_similar(positive=['python','python'], topn=5)
print(f'vocabulary size: {w2v_model.wv.vectors.shape}')
#w2v_model.wv.key_to_index.keys() #  words learnt

vocabulary size: (46783, 300)


In [25]:
def sentence2Vec(x):
    sum=np.zeros(shape=(w2v_size,), dtype=float)
    for word in x:
        if word in w2v_model.wv.key_to_index.keys():
            sum+=w2v_model.wv[word]
        
    return sum

In [26]:
X_train = X_train.apply(sentence2Vec)
X_val = X_val.apply(sentence2Vec)

In [27]:
X_train

11855    [-11.771240472793579, -15.152553103864193, -27.617887877859175, -9.939977053552866, 14.133790597319603, -9.313298674300313, -10.836395336315036, -17.799319379031658, -2.4157352969050407, -15.94972...
63955    [13.058725936338305, 12.901569683104753, 17.753930203150958, 10.553578093647957, -0.05709996074438095, -10.248427394777536, -4.590606007725, -34.10532204806805, 6.179462884552777, -14.406759255100...
57931    [4.225326258689165, 6.619971051812172, -4.598191998433322, -6.307824179530144, -6.604692071676254, 2.1905617602169514, -15.604895390570164, -19.5350179951638, -9.50810008123517, -14.27948206663131...
64857    [10.928472986444831, -1.7770422575995326, -0.7221427410840988, -1.3425220288336277, 4.400133825838566, 3.616092184558511, -6.186429604887962, -10.464931458234787, -2.3366588298231363, -0.580533226...
39463    [5.1191253662109375, -4.8990762531757355, -7.9826939925551414, 1.2017227858304977, 2.4907878624508157, -2.3661383129656315, 2.0452920272946358, -0.08836524

In [28]:
my_list_of_lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
my_array = np.asarray(my_list_of_lists)
print(my_array.shape)

(3, 3)


# Models

## functions W2V

In [29]:
def W2V(X_train):
    Xtrain = X_train.to_list()
    w2v_size=300
    w2v_window=5
    w2v_min_count=1
    w2v_epochs=100
    maxlen = 24 # adapt to length of sentences
    sentences = Xtrain
    print("Build & train Word2Vec model ...")
    w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=3)
#                                               workers=multiprocessing.cpu_count())
    w2v_model.build_vocab(sentences)
    w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
    model_vectors = w2v_model.wv
    w2v_words = model_vectors.index_to_key
    print(f"Vocabulary size: {len(w2v_words)}")
    print("Word2Vec trained")
    W2VDic = {'w2v_size':w2v_size,  
              'w2v_window':w2v_window,
              'w2v_min_count': w2v_min_count,
              'w2v_epochs':w2v_epochs,
              'w2v_model':w2v_model,
              'w2v_words':w2v_words}
    return W2VDic
              
    
def sentence2Vec(x,w2v_model,w2v_size):
    sum=np.zeros(shape=(w2v_size,), dtype=float)
    for word in x:
        if word in w2v_model.wv.key_to_index.keys():
            sum+=w2v_model.wv[word]
            
    sum=[i for i in sum] # allow to transform X_train/X_val to a numpy ndarray
    return sum

### create Data sets

In [30]:
X_train, X_val, y_train, y_val = splitData(data,XCol='tokens_lemma',Ycol='filtered_tags_reduced',ratioToKeep=1) # too much time without sampling
#print(y_train)
W2VDic =W2V(X_train)


data used shape: (51904, 20)
X_train types:	<class 'pandas.core.series.Series'>
X_test types:	<class 'pandas.core.series.Series'>
Y_test types:	<class 'pandas.core.series.Series'>
Y_train types:	<class 'pandas.core.series.Series'>
Build & train Word2Vec model ...
Vocabulary size: 69187
Word2Vec trained


In [31]:
Xtrain = X_train.apply(lambda x: sentence2Vec(x,W2VDic['w2v_model'],W2VDic['w2v_size']))
Xval = X_val.apply(lambda x: sentence2Vec(x,W2VDic['w2v_model'],W2VDic['w2v_size']))
y_train, y_val, labels = Binarize(y_train,y_val)
#print(Xtrain.shape)
Xtrain =  [i for i in Xtrain] # allow to transform X_train/X_val to a numpy ndarray
Xval = [i for i in Xval]      # allow to transform X_train/X_val to a numpy ndarray
X_train = np.asarray(Xtrain)
X_val = np.asarray(Xval)

y_train = [ i for i in y_train]
y_val = [ i for i in y_val]
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)


print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")

oversample = SMOTE(random_state=42)
X_train, y_train = oversample.fit_resample(X_train, y_train) # SMOTE
X_train, y_train = shuffle(X_train,y_train, random_state=1)
print(f"---- After SMOTE ----")
print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")

print(f"---- Reduce size training set ----")
X_train, y_train = reduceData(X_train ,y_train,ratioToKeep=0.25) #0.05

print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")
print(f"X_val vectorized shape: {X_val.shape}\n")
#np.any(y_train.sum(axis=1) > 1) # control binarize output

---------------
Ytrain vectorized shape: (34775,)

Yval vectorized shape: (17129,)

classes : ['tag0' 'tag1' 'tag10' 'tag108' 'tag11' 'tag12' 'tag13' 'tag14' 'tag15'
 'tag16' 'tag17' 'tag18' 'tag19' 'tag2' 'tag20' 'tag21' 'tag22' 'tag23'
 'tag24' 'tag25' 'tag26' 'tag27' 'tag28' 'tag29' 'tag3' 'tag30' 'tag31'
 'tag32' 'tag33' 'tag34' 'tag35' 'tag36' 'tag37' 'tag38' 'tag4' 'tag5'
 'tag6' 'tag7' 'tag8' 'tag9']
y_train vectorized shape: (34775, 40)

y_val vectorized shape: (17129, 40)

---- After SMOTE ----
y_train vectorized shape: (141840, 40)

X_train vectorized shape: (141840, 300)

---- Reduce size training set ----
y_train vectorized shape: (35460, 40)

y_val vectorized shape: (17129, 40)

X_train vectorized shape: (35460, 300)

X_val vectorized shape: (17129, 300)



In [32]:
y_train.sum(axis=0)

array([925, 913, 925, 878, 839, 915, 910, 904, 881, 925, 850, 911, 917,
       893, 873, 840, 872, 855, 880, 910, 869, 882, 855, 873, 828, 870,
       898, 837, 918, 902, 903, 847, 849, 940, 908, 886, 890, 884, 877,
       928])

###  logistic regression

In [33]:
#y_train, y_val, labels = Binarize(y_train,y_val)

log_clf = OneVsRestClassifier(LogisticRegression(random_state=0,
                                                 max_iter=10000,
                                                 verbose=0))

model = Pipeline(steps=[("scaler" , StandardScaler()),('estimator',log_clf)])
#raise Exception("trop long") 
# hyperparameters
param_grid = [{'estimator__estimator__C': [0.1, 0.5, 1., 5, 10]}] # lambda regularization

# cross-validation
clf = GridSearchCV(model,
                   param_grid,
                   scoring='f1_macro', #F1 = 2 * (precision * recall) / (precision + recall)
                   n_jobs=-1,
                   verbose=0,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train, y_train)
log_clf_best_param = clf.best_params_['estimator__estimator__C']
print('Meilleurs hyper-paramètres :')
print(clf.best_params_)

Meilleurs hyper-paramètres :
{'estimator__estimator__C': 10}


In [34]:
log_clf = OneVsRestClassifier(LogisticRegression(random_state=42,
                                                 max_iter=10000,
                                                 verbose=0,
                                                C=log_clf_best_param))
model = Pipeline(steps=[("scaler" , StandardScaler()),('estimator',log_clf)])
model.fit(X_train, y_train)
clfScore(model, X_val,y_val, labels,'W2V_logistic_reduced_without_empty')

Tags                     		 F-score		 precision		 recall

tag0                     		 0.361999		 0.235225		 0.785156
tag1                     		 0.188270		 0.113826		 0.544160
tag10                    		 0.438007		 0.473684		 0.407328
tag108                   		 0.405963		 0.343023		 0.497191
tag11                    		 0.268861		 0.223235		 0.337931
tag12                    		 0.745149		 0.742268		 0.748052
tag13                    		 0.201626		 0.180758		 0.227941
tag14                    		 0.386364		 0.411765		 0.363914
tag15                    		 0.246479		 0.214067		 0.290456
tag16                    		 0.263158		 0.284281		 0.244957
tag17                    		 0.280587		 0.315018		 0.252941
tag18                    		 0.254083		 0.303030		 0.218750
tag19                    		 0.381625		 0.473684		 0.319527
tag2                     		 0.346316		 0.237203		 0.641326
tag20                    		 0.277895		 0.273859		 0.282051
tag21                    		 0.288210		 0.318841		 0.26294

### SVC

## Random Forest

In [35]:
RF_clf = OneVsRestClassifier(RF(min_samples_split=5))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',RF_clf)])
#raise Exception("trop long") 
# hyperparameters
param_grid = [{'estimator__estimator__n_estimators': [10,20,30],'estimator__estimator__max_depth' : [5,8,10]}]

# cross-validation
clf = GridSearchCV(model,
                   param_grid,
                   scoring='f1_macro', #F1 = 2 * (precision * recall) / (precision + recall)
                   n_jobs=-1,
                   verbose=0,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train, y_train)
RF_clf_best_param1 = clf.best_params_['estimator__estimator__n_estimators']
RF_clf_best_param2 = clf.best_params_['estimator__estimator__max_depth']
print('Meilleurs hyper-paramètres :')
print(clf.best_params_)

Meilleurs hyper-paramètres :
{'estimator__estimator__max_depth': 10, 'estimator__estimator__n_estimators': 10}


In [36]:
RF_clf = OneVsRestClassifier(RF(n_estimators=RF_clf_best_param1,
                                                  max_depth=RF_clf_best_param2,
                                                   min_samples_split=5))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',RF_clf)])
model.fit(X_train, y_train)
clfScore(model, X_val,y_val, labels,'W2V_RF_reduced_without_empty')

Tags                     		 F-score		 precision		 recall

tag0                     		 0.027491		 0.014043		 0.648649
tag1                     		 0.003521		 0.001788		 0.115385
tag10                    		 0.298326		 0.245614		 0.379845
tag108                   		 0.061962		 0.034884		 0.276923
tag11                    		 0.020202		 0.011390		 0.089286
tag12                    		 0.622222		 0.469072		 0.923858
tag13                    		 0.005236		 0.002915		 0.025641
tag14                    		 0.052326		 0.031142		 0.163636
tag15                    		 0.000000		 0.000000		 0.000000
tag16                    		 0.035503		 0.020067		 0.153846
tag17                    		 0.020202		 0.010989		 0.125000
tag18                    		 0.033445		 0.021645		 0.073529
tag19                    		 0.189542		 0.127193		 0.371795
tag2                     		 0.023927		 0.012257		 0.500000
tag20                    		 0.015385		 0.008299		 0.105263
tag21                    		 0.025641		 0.014493		 0.11111

In [37]:
endTime = time.time()
print(f"Execution Time {endTime - startTime} s")

Execution Time 4908.0404715538025 s
